### Comments
* Getting the data -  I did not assume that the max peak number necessarily equal the number of peaks. Therfore I used the peak number within the HTML code from the continent webpages.
* Another point is that I preferred to get all the data and then create the dataframe, and not creating the dataframe in steps. 
* I decided to not use the pipeline function, to allow transparency of the working process, i.e. so you will be able to see the outputs of each step.
* I changed the dtypes only to the two Elevation columns since I needed that for the missing values. I did not do it for the Latitude and Longitude, I will be able to do that on the analysis phase. 

In [1]:
import re
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from tqdm import tqdm

In [2]:
URL_MAIN = 'http://www.peakware.com/peaks.php'
CONTINENT_CODE_LIST = ['AfA', 'AnA', 'AsA', 'AuA', 'EuA', 'NoA', 'SoA', 'ABC']
METER_TO_FEET = 3.28084
feet_to_meter = 1 / METER_TO_FEET

In [8]:
class Peak():
    def __init__(self, peak_name, peak_num):
        self.name = peak_name
        self.peak_num = peak_num
        
        self.features = {} # This dictionary contains the peak's features. Empty upon instance creation
    
    # The following method receives the main URL and uses the peak number to get into the peak's webpage, and pull all the 
    # attributs of the peak, and store it in the self.features dictionary
    def get_peak_features(self, url):
        peak_url = "{}?pk={}".format(url, self.peak_num) # Gets the URL of the peak
        peak_resp = requests.get(peak_url) 
        peak_soup = bs(peak_resp.text, 'lxml')
        main = peak_soup.find(name = 'main') # Filters the soup only to the 'main' tag
        peak_features = main.find_all(name = 'tr') # creates a list with all the 'tr' tags within 'main'
        # The following for loop runs on each 'tr' tag, finds for each feature its name (key) and value
        for tr in peak_features:
            feature = tr.th.text[:-1] # Takes out the colon at the end
            self.features[feature] = tr.td.text # I can turn this into an integer now or later

In [4]:
def get_name_and_num_for_all_peaks(url, continent_code_list):
    """
    The function brings all the numbers and names of the peaks, running on continent web pages one after the other.
    Inputs:
    1. url - the main URL the precedes the scepefic code for the peak
    2. continent_code_list - a list of continent codes
    The function returns a list of tuples, each tuple contains the peak's name and number (site code for the peak)
    """
    
    list_of_peaks = [] # The tuples will be stored within this list
    # The following for loop brings the function output per continent
    for short_cont_name in tqdm(continent_code_list):
        continent_url = "{}?choice={}".format(url, short_cont_name) # Gets the URL of the continent (order by names)
        continent_resp = requests.get(continent_url)
        continent_soup = bs(continent_resp.text, 'lxml')
        peak_list = continent_soup.find(id="peakList") # Filters the soup only to the tag with id="peakList"
        li_peaks = peak_list.find_all(name='li') # Each peak is bounded by the 'li' tag
        # The following for loop runs on each peak and brings the name and number
        for li_peak in li_peaks:
            peak_name = li_peak.a.text
            peak_num = re.findall('\d+', str(li_peak.a)) # Using regex to get the number
            list_of_peaks.append((peak_name, int(peak_num[0])))
    return list_of_peaks

In [5]:
peak_tuple_list = get_name_and_num_for_all_peaks(URL_MAIN, CONTINENT_CODE_LIST)

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:12<00:00,  1.85s/it]


In [6]:
len(peak_tuple_list) # making sure that the number makes sense

4194

In [9]:
# Test to see that it extracts the features - Just a test, not part of the main code

# peak1 = Peak(peak_tuple_list[2][0], peak_tuple_list[2][1])
# print(peak1.features) # prints before getting the features
# peak1.get_peak_features(URL_MAIN)
# print(peak1.features) # prints after getting the features

{}
{'Elevation (feet)': '11,864', 'Elevation (meters)': '3,616', 'Continent': 'Africa', 'Country': 'Morocco', 'Range/Region': 'High Atlas', 'Latitude': '31.1753', 'Longitude': '-7.8254', 'Difficulty': 'Scramble', 'Best months for climbing': 'Jan, Feb, Mar, Apr, May, Jun, Oct, Nov, Dec', 'Nearest major airport': 'Marrakech', 'Convenient Center': 'Marrakech'}


In [10]:
def make_df_from_peak_list(peak_list, url):
    """
    The fuction takes a list of tuples with the peaks' names and numbers and returns a dataframe with the data.
    Input: peak_list - a list of tuples with the peak's name and number
    The function does the following:
    1. creates an instance of Peak
    2. gets features from the peak's webpage
    3. puts everything in a dicitonary
    3. turns the dictionary into a dataframe
    The function returns a dataframe with all peaks and their features
    """
    
    dict_of_peaks = {} # The features will be stored within this dict
    for tup in tqdm(peak_list):
        #I had a problem with the SSL verification so I am using this try-except expression 
        try:
            peak_instance = Peak(tup[0], tup[1]) # Create a peak instance
            peak_instance.get_peak_features(url) # Get the featres - using the class method
            dict_of_peaks[peak_instance.name] = peak_instance.features # Saving the features into the dict
        except:
            print("could not get details for peak no. {}.".format(tup[1]))
    data = pd.DataFrame.from_dict(dict_of_peaks) # creating a pandas dataframe
    data = data.T
    return data

In [11]:
data = make_df_from_peak_list(peak_tuple_list, URL_MAIN) # Creating the dataframe by calling the make_df_from_peak_list function

100%|████████████████████████████████████████████████████████████████████████████| 4194/4194 [2:02:47<00:00,  1.76s/it]


In [12]:
# At this point I have a raw dataframe with all the peaks' features
data.head()

,Best months for climbing,Continent,Convenient Center,Country,Difficulty,Elevation (feet),Elevation (meters),First successful climber(s),Latitude,Longitude,Most recent eruption,Nearest major airport,Province,Range/Region,State,Volcanic status,Year first climbed
"19th Party Conference, Peak","Jun, Jul, Aug",Asia,Osh,Kyrgyzstan,Basic Snow/Ice Climb,"5,885","1,794",Vitaly Abalakow,39.3833,72.9667,NaN,Osh,NaN,Pamir,NaN,NaN,1930
"8620, Peak","Apr, May, Jun",North America,Denali Park,United States,Technical Climb,"8,620","2,627",NaN,63.25,-150.25,NaN,Fairbanks,NaN,Alaska Range,Alaska,NaN,NaN
A'Mhaighdean,"May, Jun, Jul",Europe,Inverness,United Kingdom,Scramble,"3,173",967,NaN,57.7196,-5.3463,NaN,Inverness,NaN,Scottish Highlands,NaN,NaN,NaN
"Abbot, Mount","Jun, Jul, Aug, Sep",North America,Toms Place or Bishop,United States,Basic Snow/Ice Climb,"13,704","4,177","Jame Hutchinson, Joseph LeConte and Duncan McD...",37.3864,-118.784,NaN,NaN,NaN,Sierra Nevada,California,NaN,1908
Abbott Butte,"May, Jun, Jul, Aug, Sep, Oct",North America,"Union Peak, Oregon",United States,NaN,"6,125","1,867",NaN,42.943457,-122.549764,NaN,Eugene Airport,NaN,Cascade Range,Oregon,NaN,NaN


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4092 entries, 19th Party Conference, Peak to Šiljak (Rtanj)
Data columns (total 17 columns):
Best months for climbing       3834 non-null object
Continent                      4092 non-null object
Convenient Center              3813 non-null object
Country                        4092 non-null object
Difficulty                     3841 non-null object
Elevation (feet)               4086 non-null object
Elevation (meters)             4083 non-null object
First successful climber(s)    1470 non-null object
Latitude                       3500 non-null object
Longitude                      3500 non-null object
Most recent eruption           111 non-null object
Nearest major airport          3775 non-null object
Province                       248 non-null object
Range/Region                   3195 non-null object
State                          1182 non-null object
Volcanic status                230 non-null object
Year first climbed             14

In [14]:
#I am checking that the data mekes sense in terms of how many unique values in each column
data.nunique()

Best months for climbing        290
Continent                         7
Convenient Center              2229
Country                         196
Difficulty                        7
Elevation (feet)               3194
Elevation (meters)             2747
First successful climber(s)    1325
Latitude                       3380
Longitude                      3414
Most recent eruption             96
Nearest major airport          1482
Province                          9
Range/Region                    237
State                            60
Volcanic status                   3
Year first climbed              419
dtype: int64

In [15]:
# I was asked to turn the data to lowercase. I prefer to do that at the beginning of the eprocessing since it turns all 
# the dtypes into strings. Any way I am doing it only to the columns where terms may be written differently for different peaks
list_columns_lowercase = ['Continent', 'Country', 'Difficulty', 'Nearest major airport']
data.loc[:, list_columns_lowercase] = data.loc[:, list_columns_lowercase].apply(lambda x: x.astype(str).str.lower())
# I was also asked to turn the missing values to None. Since None gives me NaN, I am leaving the null values as is.

In [16]:
# Checking that it has changed to lowercase
data.head()

,Best months for climbing,Continent,Convenient Center,Country,Difficulty,Elevation (feet),Elevation (meters),First successful climber(s),Latitude,Longitude,Most recent eruption,Nearest major airport,Province,Range/Region,State,Volcanic status,Year first climbed
"19th Party Conference, Peak","Jun, Jul, Aug",asia,Osh,kyrgyzstan,basic snow/ice climb,"5,885","1,794",Vitaly Abalakow,39.3833,72.9667,NaN,osh,NaN,Pamir,NaN,NaN,1930
"8620, Peak","Apr, May, Jun",north america,Denali Park,united states,technical climb,"8,620","2,627",NaN,63.25,-150.25,NaN,fairbanks,NaN,Alaska Range,Alaska,NaN,NaN
A'Mhaighdean,"May, Jun, Jul",europe,Inverness,united kingdom,scramble,"3,173",967,NaN,57.7196,-5.3463,NaN,inverness,NaN,Scottish Highlands,NaN,NaN,NaN
"Abbot, Mount","Jun, Jul, Aug, Sep",north america,Toms Place or Bishop,united states,basic snow/ice climb,"13,704","4,177","Jame Hutchinson, Joseph LeConte and Duncan McD...",37.3864,-118.784,NaN,nan,NaN,Sierra Nevada,California,NaN,1908
Abbott Butte,"May, Jun, Jul, Aug, Sep, Oct",north america,"Union Peak, Oregon",united states,nan,"6,125","1,867",NaN,42.943457,-122.549764,NaN,eugene airport,NaN,Cascade Range,Oregon,NaN,NaN


In [17]:
# I am leaving only the columns I need
data = data[['Continent', 'Country', 'Difficulty', 'Elevation (feet)', 'Elevation (meters)', 'Latitude', 'Longitude',\
                    'Nearest major airport', 'Year first climbed']]

In [18]:
# This step takes out the comma from the Elevation columns
elevation_columns_no_comma = data.loc[:,['Elevation (feet)', 'Elevation (meters)']].apply(lambda x: x.str.replace(",",""))
data.loc[:,['Elevation (feet)', 'Elevation (meters)']] = elevation_columns_no_comma

In [19]:
# I am making sure that I have the columns that I need and that the commas were taken out
data.head()

,Continent,Country,Difficulty,Elevation (feet),Elevation (meters),Latitude,Longitude,Nearest major airport,Year first climbed
"19th Party Conference, Peak",asia,kyrgyzstan,basic snow/ice climb,5885,1794,39.3833,72.9667,osh,1930
"8620, Peak",north america,united states,technical climb,8620,2627,63.25,-150.25,fairbanks,NaN
A'Mhaighdean,europe,united kingdom,scramble,3173,967,57.7196,-5.3463,inverness,NaN
"Abbot, Mount",north america,united states,basic snow/ice climb,13704,4177,37.3864,-118.784,nan,1908
Abbott Butte,north america,united states,nan,6125,1867,42.943457,-122.549764,eugene airport,NaN


In [20]:
# In order to be able to fill in the missing values of the Elevation columns by calculation, I need to turn the values in
# these colums into numeric ones
for col in ['Elevation (feet)', 'Elevation (meters)']:
    data.loc[:, col] = pd.to_numeric(data.loc[:, col], errors='coerce') # Using 'coerce' since I am reassigning the output
# checking dtypes
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4092 entries, 19th Party Conference, Peak to Šiljak (Rtanj)
Data columns (total 9 columns):
Continent                4092 non-null object
Country                  4092 non-null object
Difficulty               4092 non-null object
Elevation (feet)         4086 non-null float64
Elevation (meters)       4083 non-null float64
Latitude                 3500 non-null object
Longitude                3500 non-null object
Nearest major airport    4092 non-null object
Year first climbed       1461 non-null object
dtypes: float64(2), object(7)
memory usage: 319.7+ KB


In [21]:
# For peaks that are missing both the Elevation by feet and by meters, I am taking these out (using the ~ sign)
data = data[~(data['Elevation (feet)'].isnull() & data['Elevation (meters)'].isnull())] 

In [22]:
# The following lines fill in the missing values in the Elevation columns
# null_elevation_feet are the indices of the null values in 'Elevation (feet)'
null_ealevation_feet = data['Elevation (feet)'][data['Elevation (feet)'].isnull()].index # These are the indices of the lines
data.loc[null_ealevation_feet, 'Elevation (feet)'] = data['Elevation (meters)'] * METER_TO_FEET

# null_elevation_meters are the indices of the null values in 'Elevation (meters)'
null_ealevation_meters = data['Elevation (meters)'][data['Elevation (meters)'].isnull()].index
data.loc[null_ealevation_meters, 'Elevation (meters)'] = data['Elevation (feet)'] * feet_to_meter

In [23]:
# After we take out the nans we can turn it into int
data[['Elevation (feet)', 'Elevation (meters)']] = data[['Elevation (feet)', 'Elevation (meters)']].astype(int)

In [24]:
# Making sure that I don't have any null in the Elavation colums as before 
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4086 entries, 19th Party Conference, Peak to Šiljak (Rtanj)
Data columns (total 9 columns):
Continent                4086 non-null object
Country                  4086 non-null object
Difficulty               4086 non-null object
Elevation (feet)         4086 non-null int32
Elevation (meters)       4086 non-null int32
Latitude                 3499 non-null object
Longitude                3499 non-null object
Nearest major airport    4086 non-null object
Year first climbed       1461 non-null object
dtypes: int32(2), object(7)
memory usage: 447.3+ KB


In [25]:
# The following line takes values of the 'Country' columns with two countries, separated by '/', split it by '/' and takes the
# first country
data['Country'] = data['Country'].str.split("/").str.get(0)

In [26]:
# The line below is to examine what types of texts I may find in the 'Year first climbed' and is not part of the final code.
# I did it several times to examine different types
# data['Year first climbed'][data['Year first climbed'].notnull()]

19th Party Conference, Peak                        1930
Abbot, Mount                                       1908
Aberdeen, Mount                                    1894
Aconcagua                                          1897
Adamello                                           1864
Adi Kailash                                        2004
Agassiz, Mount                                     1925
Agios Bernados                                     1800
Agnes Mountain                                     1936
Ago di Tredenus                                    1932
Aguglia di Goloritzè                               1981
Aguja De La S                                      1968
Aguja Saint Exupery                                1968
Aiguille Du Midi                                   1911
Aiguille Du Moine                                  2001
Aiguille Du Tour                                   1926
Aiguille Sans Nom                                  1898
Aiguille Verte                                  

In [27]:
def extract_year_from_text(text):
    """
    The function takes a text and returns a 4 digit year, based on regex or using to_datetime. If doesn't find returns None
    Input: text - any text
    Returns an integer of the year or None
    """
    
    if type(text) == str: # The NaNs are floats, and I must use string for the regex
        if re.findall('\d{4}', text) != []: # If the regex finds a year
            year_str_list = re.findall('\d{4}', text) # Asuming no records prior to the year 1000 (3 digits)
            # Since there are values with two years, I am turning it to integer, and taking the minimum (foloowing 2 lines)
            year_int_list = [int(one_year) for one_year in year_str_list]
            return str(min(year_int_list))
        else:
            try:
                time_stamp = pd.to_datetime(text) # Aims mainly for values where the year is represented in two digits 
                return str(time_stamp.year)
            except:
                return None # If not a string change back to None

In [28]:
# The following line applies the extract_year_from_text function to the values in column 'Year first climbed'
data.loc[:,'Year first climbed'] = data.loc[:,'Year first climbed'].apply(extract_year_from_text) 

In [29]:
data.to_csv("{project_#2_tools_data}_{ishay_telavivi}.csv", index_label='Peak name')